In [43]:
# prompt: check GPU
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [44]:
import torch
import numpy as np
import cv2
import os
from detect import run
from models.common import DetectMultiBackend
from models.classification import Classification_model
from utils.torch_utils import select_device
ROOT = os.getcwd() 

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Demo

## Input:

In [46]:
ROOT = os.getcwd() 
file_output_dir = os.path.join(ROOT, 'Demo', 'annotation.txt') 
video_output_dir = os.path.join(ROOT, 'Demo', 'video_output.mp4')
video_dir = r'E:\App\Git\repos\yolov9-face-detection\yolov9\WIN_20240610_18_31_30_Pro.mp4'

## Tách từng frame và xử lí

In [47]:
class Params:
    def __init__(self):
        self.detection_weights = os.path.join(ROOT, 'detection_weights.pt')
        self.classification_weights = os.path.join(ROOT, 'classification_weights.pt')
        self.processing_dir = os.path.join(ROOT, 'Demo')
        self.detection_name = 'exp'
        self.boxes_dir = os.path.join(self.processing_dir, self.detection_name, 'labels', 'frame.txt') 
        self.data = os.path.join(ROOT, 'data', 'coco.yaml')


        self.detection_model = DetectMultiBackend(self.detection_weights, device=device, dnn=False, data=self.data, fp16=False)
        self.classification_model = Classification_model(self.classification_weights, num_classes=7)

params = Params()

Fusing layers... 
yolov9-c summary: 604 layers, 50698278 parameters, 0 gradients, 236.6 GFLOPs


In [48]:
def delete_file(folder_path):
    # Kiểm tra nếu đường dẫn là một tệp
    if os.path.isfile(folder_path):
        os.remove(folder_path)

    # Kiểm tra nếu đường dẫn là một thư mục
    elif os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            delete_file(file_path)
        os.rmdir(folder_path)

In [49]:
def detection(source, frame):
      #command = f'python {params.detection} --weights {params.weights} --source {source} --project {params.processing_dir} --name {params.detection_name} --save-txt --nosave'
      run(model = params.detection_model, weights= params.detection_weights, source=source, project=params.processing_dir, name = params.detection_name, save_txt = True, nosave= True)
      boxes_dir = params.boxes_dir

      # đọc file boundingbox vừa được tạo ra
      with open(boxes_dir, 'r') as file:
          lines = file.readlines()

      # Tách từng dòng và xử lý từng hàng
      processed_lines = []
      for line in lines:
          # Tách các giá trị trong dòng, bỏ qua các khoảng trắng không cần thiết
          values = line.strip().split()
          # Chuyển các giá trị từ chuỗi sang kiểu float (nếu cần)
          float_values = [float(value) for value in values]
          x, y, w, h = float_values[1:5]
          x, y, w, h = map(int, [x * frame.shape[1], y * frame.shape[0],
                      w * frame.shape[1], h * frame.shape[0]])
          processed_lines.append([x, y, w, h])
      delete_file(params.processing_dir + '/' + params.detection_name)
      return processed_lines


In [50]:
def draw_bounding_boxes(frame, bounding_boxes, labels):
    # Vẽ bounding boxes trên frame
    for bbox,label in zip(bounding_boxes, labels):
        x, y, w, h = bbox
        x1, y1 = x - w // 2, y - h // 2
        x2, y2 = x1 + w, y1 + h
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'{label}', (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    return frame

In [51]:
def cut_image(frame, bounding_boxes, gain = 1.02, pad = 10):
    face_images = []
    for bbox in bounding_boxes:
        x, y, w, h = bbox
        x1, y1 = x - w // 2, y - h // 2
        x2, y2 = x1 + w, y1 + h
        crop = frame[y1:y2, x1:x2]
        crop = cv2.resize(crop, (100, 100))
        face_images.append(crop)
    return face_images

In [52]:
def classification(model, face_images):
    class_labels = np.array(['Surprise', 'Fear', 'Disgust', 'Happy', 'Sad', 'Angry', 'Neutral'])
    x_tensor = torch.tensor(face_images, dtype=torch.float32).permute(0, 3, 1, 2).to(select_device(''))
    output = model(x_tensor)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    predicted_classes = torch.argmax(probabilities, dim=1)

    label_sto = []
    for i in range(len(predicted_classes)):
        label_sto.append(class_labels[predicted_classes[i].item()])
    return label_sto

In [53]:
def write_to_annotation_file(file_path, bounding_boxes, labels, frame_idx):
    with open(file_path, 'a') as file:
        for bbox, label in zip(bounding_boxes, labels):
            x, y, w, h = bbox
            file.write(f"{frame_idx} {x} {y} {w} {h} {label}\n")

In [54]:
def create_video_output(video, video_output_dir): 
    fps = video.get(cv2.CAP_PROP_FPS)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Khởi tạo đối tượng VideoWriter để tạo video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Xác định codec
    return cv2.VideoWriter(video_output_dir, fourcc, fps, (width, height)) 

In [55]:
def main(video_dir, file_output_dir:str = os.path.join(ROOT, 'Demo', 'annotation.txt'), video_output_dir: str = os.path.join(ROOT, 'Demo', 'video_output.mp4')):
    
    # Đọc và lấy số frame của video hiện tại
    video = cv2.VideoCapture(video_dir)
    num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    out = create_video_output(video, video_output_dir)
        
    for frame_idx in range(num_frames):
        if (frame_idx % 10 == 0):
            print(f"Processing frame {frame_idx} of {num_frames}")
        # Đọc frame
        ret, frame = video.read()
        if not ret:
            break
        frame_path = os.path.join(params.processing_dir, 'frame' + '.jpg')
        cv2.imwrite(frame_path, frame)

        # Chạy detection
        bounding_boxes = detection(frame_path, frame)

        #Cắt cận ảnh khuôn mặt và resize
        face_images = cut_image(frame, bounding_boxes)

        #Phân loại cảm xúc
        labels = classification(params.classification_model, face_images)

        #Ghi lại thông tin vào file annotation.txt
        write_to_annotation_file(file_output_dir, bounding_boxes, labels, frame_idx)

        #Vex bounding boxes và labels cho frame hiện tại
        drawn_frame = draw_bounding_boxes(frame, bounding_boxes, labels)

        #Thêm frame đã được vẽ bounding boxes và labels vào video output
        out.write(frame)
        
        # Frame sau khi xử lí xong
        os.remove(frame_path)

    # Đóng video
    video.release()
    out.release()

    print(f"Bounding boxes and labels saved to {file_output_dir}")
    print(f"Results video saved to {video_output_dir}")


In [56]:
if __name__ == "__main__":
    video_dir = r'E:\App\Git\repos\yolov9-face-detection-Copy\yolov9\WIN_20240612_10_54_30_Pro.mp4'
    main(video_dir)

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU



Processing frame 0 of 65


YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cp

Processing frame 10 of 65


YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cp

...

Processing frame 50 of 65


YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cp

Processing frame 60 of 65


YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU

YOLOv5  2024-6-11 Python-3.12.2 torch-2.3.0+cpu CPU



Bounding boxes and labels saved to e:\App\Git\repos\yolov9-face-detection-Copy\yolov9\Demo\annotation.txt
Results video saved to e:\App\Git\repos\yolov9-face-detection-Copy\yolov9\Demo\video_output.mp4
